In [ ]:
!pip install -U langchain langchain-community langchain-openai langchain-core -q
!pip install pymupdf faiss-cpu tiktoken chromadb pypdf unstructured[all] pdf2image pytesseract -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1. 문서 업로드 및 텍스트 추출

In [ ]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

print(pdf_path)

Saving 노인맞춤돌봄서비스.pdf to 노인맞춤돌봄서비스.pdf
노인맞춤돌봄서비스.pdf


In [ ]:
'''from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_path)
documents = loader.load()

documents''' # PyPDFLoader의 깨짐 현상으로 fitz로 텍스트 추출 후 document 객체 생성

'from langchain.document_loaders import PyPDFLoader\n\nloader = PyPDFLoader(pdf_path)\ndocuments = loader.load()\n\ndocuments'

In [ ]:
from langchain.schema import Document
import fitz  # PyMuPDF

def load_pdf_with_fitz_and_metadata(pdf_path):
    doc = fitz.open(pdf_path)
    documents = []
    for i, page in enumerate(doc):
        text = page.get_text("text").strip()

        metadata = {
            "source": pdf_path,
            "page": i,
            "total_pages": len(doc),
            "page_label": page.get_label() if hasattr(page, "get_label") else str(i + 1)
        }

        documents.append(Document(page_content=text, metadata=metadata))
    return documents

documents = load_pdf_with_fitz_and_metadata(pdf_path)
documents[0]

Document(metadata={'source': '노인맞춤돌봄서비스.pdf', 'page': 0, 'total_pages': 2, 'page_label': ''}, page_content='1. 서비스 대상자 선정 \n \n가. 서비스 신청 \n• \n노인맞춤돌봄서비스를 필요로 하는 노인 또는 그 가족 등이 읍·면·동 행정복지센\n터 방문 등을 통해 서비스 신청 \n \n▣ 서비스 신청자격 \n• \n65세 이상 국민기초생활수급자, ②차상위계층 또는 ③기초연금수급자로서 유사중\n복사업 자격에 해당되지 않는 자 \n(단, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 제공 가\n능) \n \n▣ 유사중복사업 자격 해당자 \n※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌\n봄서비스 보다 우선적으로 제공 \n1. 노인장기요양보험 등급자 \n※ 노인장기요양보험은 노인맞춤돌봄서비스의 선순위로서 장기요양 등급자가 장\n기요양을 포기하고 노인맞춤돌봄서비스를 신청하는 것은 불가함 \n① 장기요양 등급 유효기간 만료자(신청 가능하고 노인장기요양보험 등급고지가 \n요청되고, \n시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 신청 가능) \n※ 신청 당시 본인 질병·사유 등으로 장기요양 신청이 어려운 경우, 해당 사유를 \n기재한 장기요양정보제공신청서(행복e음)를 통해 확인 시 노인맞춤돌봄서비스 제\n공(장기요양보험정보제공서 제출) \n2. 가사·간병 방문지원사업 이용자 \n3. 국가보훈처 보훈재가복지서비스 이용자 \n4. 기타 유사 돌봄서비스 이용자 \no \n기타 국가 및 지방자치단체(이하 “지자체”)에서 시행하는 서비스 중 노인맞\n춤돌봄서비스와 유사한 재가서비스 \n※ 시·군·구는 타 사업의 특성, 실제적 제공 내용 및 돌봄 필요성 등을 고려')

## 2. 텍스트 분할

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""] # chunk_size 이후 구분자(\n\n)가 나타나면 더 작은 구분자(\n)로 나눔
)

split_docs = text_splitter.split_documents(documents)

print(f"총 청크 개수: {len(split_docs)}")
for i, doc in enumerate(split_docs):
    print(f'\n[문서 {i+1}]\n{doc.page_content}')
    if i == 2:
        break


총 청크 개수: 5

[문서 1]
1. 서비스 대상자 선정 
 
가. 서비스 신청 
• 
노인맞춤돌봄서비스를 필요로 하는 노인 또는 그 가족 등이 읍·면·동 행정복지센
터 방문 등을 통해 서비스 신청 
 
▣ 서비스 신청자격 
• 
65세 이상 국민기초생활수급자, ②차상위계층 또는 ③기초연금수급자로서 유사중
복사업 자격에 해당되지 않는 자 
(단, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 제공 가
능) 
 
▣ 유사중복사업 자격 해당자 
※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌

[문서 2]
능) 
 
▣ 유사중복사업 자격 해당자 
※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌
봄서비스 보다 우선적으로 제공 
1. 노인장기요양보험 등급자 
※ 노인장기요양보험은 노인맞춤돌봄서비스의 선순위로서 장기요양 등급자가 장
기요양을 포기하고 노인맞춤돌봄서비스를 신청하는 것은 불가함 
① 장기요양 등급 유효기간 만료자(신청 가능하고 노인장기요양보험 등급고지가 
요청되고, 
시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 신청 가능)

[문서 3]
① 장기요양 등급 유효기간 만료자(신청 가능하고 노인장기요양보험 등급고지가 
요청되고, 
시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 신청 가능) 
※ 신청 당시 본인 질병·사유 등으로 장기요양 신청이 어려운 경우, 해당 사유를 
기재한 장기요양정보제공신청서(행복e음)를 통해 확인 시 노인맞춤돌봄서비스 제
공(장기요양보험정보제공서 제출) 
2. 가사·간병 방문지원사업 이용자 
3. 국가보훈처 보훈재가복지서비스 이용자 
4. 기타 유사 돌봄서비스 이용자 
o


## 3. 임베딩 생성 + 벡터 DB 구축 (Chroma 사용)

In [ ]:
pip install -U langchain-huggingface -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 한국어 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts") # BM-K/KoSimCSE-roberta

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
'''texts = [doc.page_content for doc in split_docs]
embeddings = embedding_model.embed_documents(texts)  # 임베딩 벡터 직접 생성(chunk 단위)

print(len(embeddings))
print(len(embeddings[0])) # 768차원
print(embeddings[0][:5]) # 벡터 일부 출력'''

'texts = [doc.page_content for doc in split_docs]\nembeddings = embedding_model.embed_documents(texts)  # 임베딩 벡터 직접 생성(chunk 단위)\n\nprint(len(embeddings))\nprint(len(embeddings[0])) # 768차원\nprint(embeddings[0][:5]) # 벡터 일부 출력'

In [ ]:
pip install -U langchain-chroma -q

In [ ]:
import os
from langchain_chroma import Chroma
from hashlib import md5

def file_hash(path):
    with open(path, 'rb') as f:
        return md5(f.read()).hexdigest()

def get_saved_hash(hash_path):
    if os.path.exists(hash_path):
        with open(hash_path, 'r') as f:
            return f.read()
    return None

def save_hash(hash_path, hash_value):
    with open(hash_path, 'w') as f:
        f.write(hash_value)

def check_or_build_vectorstore(pdf_path, split_docs, embedding_model,
                               db_path="./chroma_db", hash_path="./pdf.hash"):
    current_hash = file_hash(pdf_path)
    saved_hash = get_saved_hash(hash_path)

    if os.path.exists(db_path) and os.path.isdir(db_path) and current_hash == saved_hash:
        vectorstore = Chroma(
            persist_directory=db_path,
            embedding_function=embedding_model
        )
        print("기존 Chroma DB 불러오기 완료")
    else:
        vectorstore = Chroma.from_documents(
            documents=split_docs,
            embedding=embedding_model,
            persist_directory=db_path
        )
        save_hash(hash_path, current_hash)
        print("새로운 Chroma DB 생성 및 해시 저장 완료")

    return vectorstore

In [ ]:
vectorstore = check_or_build_vectorstore(pdf_path, split_docs, embedding_model)

새로운 Chroma DB 생성 및 해시 저장 완료


## 4. Retriever 구성

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "lambda_mult": 0.5}
)

In [ ]:
docs = retriever.invoke("차상위 조건이 뭐야?")

for i, doc in enumerate(docs):
    print(f'\n[문서 {i+1}]\n{doc.page_content}')


[문서 1]
능) 
 
▣ 유사중복사업 자격 해당자 
※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌
봄서비스 보다 우선적으로 제공 
1. 노인장기요양보험 등급자 
※ 노인장기요양보험은 노인맞춤돌봄서비스의 선순위로서 장기요양 등급자가 장
기요양을 포기하고 노인맞춤돌봄서비스를 신청하는 것은 불가함 
① 장기요양 등급 유효기간 만료자(신청 가능하고 노인장기요양보험 등급고지가 
요청되고, 
시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 신청 가능)

[문서 2]
1. 서비스 대상자 선정 
 
가. 서비스 신청 
• 
노인맞춤돌봄서비스를 필요로 하는 노인 또는 그 가족 등이 읍·면·동 행정복지센
터 방문 등을 통해 서비스 신청 
 
▣ 서비스 신청자격 
• 
65세 이상 국민기초생활수급자, ②차상위계층 또는 ③기초연금수급자로서 유사중
복사업 자격에 해당되지 않는 자 
(단, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 제공 가
능) 
 
▣ 유사중복사업 자격 해당자 
※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌

[문서 3]
·판단하여 사업별 유사 여부 결정 및 서비스 제공 여부 판단 (예: ○○시 
도시재생뉴딜사업 – 유사성 해당여부, ○○시 재가서비스 – 유사성 해당/
미해당) 
 
※ 노인맞춤돌봄서비스는 장기요양등급 외 판정 노인에게 다양한 서비스를 통합적으로 
제공함으로써 노인들의 자립생활을 지원하고자 하는 예방적 서비스로, 아래의 유사중복
사업 자격 해당자는 노인맞춤돌봄서비스 신청 시 우선제외 대상임.


## 5. LLM 불러오기

In [ ]:
import os
import requests
from typing import Optional, List, ClassVar

from langchain_core.language_models import LLM


# API Key
os.environ["SOLAR_API_KEY"] = "????????????/"

# SolarLLM 정의
class SolarLLM(LLM):
    model: ClassVar[str] = "solar-1-mini-chat"
    api_url: ClassVar[str] = "https://api.upstage.ai/v1/solar/chat/completions"
    api_key: ClassVar[str] = os.environ.get("SOLAR_API_KEY")

    def _call(self, prompt, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "당신은 유능한 한국어 비서입니다."},
                {"role": "user", "content": prompt}
            ],
            "top_p": 0.9,
            "temperature": 0.7
        }

        response = requests.post(self.api_url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content'].strip()

    @property
    def _llm_type(self) -> str:
        return "solar"

## 6. Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
      당신은 전문 문서 기반 질문에 답변하는 한국어 인공지능입니다.
      아래는 참고할 수 있는 문서와 사용자의 질문입니다. 문서 내용을 기반으로 정확하고 간결하게 한국어로 답변하세요.

      [Retrieve 문서]
      {context}

      [질문]
      {question}

      [답변]
      """
)

## 7. LLMChain 생성, QA

In [ ]:
from langchain.chains import LLMChain

llm = SolarLLM()
old_qa_chain = LLMChain(llm=llm, prompt=custom_prompt)
qa_chain = custom_prompt | llm

<ipython-input-16-3270f1a5b64a>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  old_qa_chain = LLMChain(llm=llm, prompt=custom_prompt)


In [ ]:
# 사용자 질문
question = (
    "안녕하세요. 저희 어머니께서 만 65세 이상이시고, "
    "기초연금을 받고 계십니다.\n"
    "또한, 현재 혼자 지내시며 건강도 많이 안 좋으신데,\n"
    "노인맞춤돌봄서비스를 신청할 수 있을까요?"
)

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
response = old_qa_chain.invoke({"context": context, "question": question})
response

{'context': '1. 서비스 대상자 선정 \n \n가. 서비스 신청 \n• \n노인맞춤돌봄서비스를 필요로 하는 노인 또는 그 가족 등이 읍·면·동 행정복지센\n터 방문 등을 통해 서비스 신청 \n \n▣ 서비스 신청자격 \n• \n65세 이상 국민기초생활수급자, ②차상위계층 또는 ③기초연금수급자로서 유사중\n복사업 자격에 해당되지 않는 자 \n(단, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 제공 가\n능) \n \n▣ 유사중복사업 자격 해당자 \n※ 노인맞춤돌봄서비스는 예방적 돌봄서비스로, ①~⑤까지의 유사중복사업은 노인맞춤돌\n\n① 장기요양 등급 유효기간 만료자(신청 가능하고 노인장기요양보험 등급고지가 \n요청되고, \n시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 신청 가능) \n※ 신청 당시 본인 질병·사유 등으로 장기요양 신청이 어려운 경우, 해당 사유를 \n기재한 장기요양정보제공신청서(행복e음)를 통해 확인 시 노인맞춤돌봄서비스 제\n공(장기요양보험정보제공서 제출) \n2. 가사·간병 방문지원사업 이용자 \n3. 국가보훈처 보훈재가복지서비스 이용자 \n4. 기타 유사 돌봄서비스 이용자 \no\n\n공(장기요양보험정보제공서 제출) \n2. 가사·간병 방문지원사업 이용자 \n3. 국가보훈처 보훈재가복지서비스 이용자 \n4. 기타 유사 돌봄서비스 이용자 \no \n기타 국가 및 지방자치단체(이하 “지자체”)에서 시행하는 서비스 중 노인맞\n춤돌봄서비스와 유사한 재가서비스 \n※ 시·군·구는 타 사업의 특성, 실제적 제공 내용 및 돌봄 필요성 등을 고려',
 'question': '안녕하세요. 저희 어머니께서 만 65세 이상이시고, 기초연금을 받고 계십니다.\n또한, 현재 혼자 지내시며 건강도 많이 안 좋으신데,\n노인맞춤돌봄서비스를 신청할 수 있을까요?',
 'text': '네, 노인맞춤돌봄서비스를 신청할 수 있습니다. 서비스 신청자격에 해당되는 65세 이상 국민기초생활수급자, 차상위계층 또는 기초연금수급자

In [ ]:
question = (
    "안녕하세요. 저희 어머니께서 만 65세 이상이시고, "
    "기초연금을 받고 계십니다.\n"
    "또한, 현재 혼자 지내시며 건강도 많이 안 좋으신데,\n"
    "노인맞춤돌봄서비스를 신청할 수 있을까요?"
)
response = qa_chain.invoke({"context": context, "question": question})
response

'안녕하세요. 사용자님의 어머니께서는 노인맞춤돌봄서비스를 신청하실 수 있습니다. 그 이유는 65세 이상이며 기초연금 수급자 자격에 해당되기 때문입니다. 또한, 혼자 지내시며 건강이 좋지 않으신 상황으로 보아 서비스 제공이 필요한 상황으로 판단됩니다. 다만, 서비스 신청을 위해서는 어머니 또는 가족 등이 읍·면·동 행정복지센터를 방문하여 신청하셔야 합니다.'

In [ ]:
# 사용자 질문
question2 = "난 1959년생이고, 차상위계층으로 가사·간병 방문지원사업 이용자야. 노인맞춤돌봄서비스를 신청하고 싶어"

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
new_response = qa_chain.invoke({"context": context, "question": question2})

new_response

'사용자는 1959년생으로 차상위계층에 해당되며 가사·간병 방문지원사업 이용자입니다. 노인맞춤돌봄서비스의 서비스 신청자격은 65세 이상 국민기초생활수급자, 차상위계층 또는 기초연금수급자로서 유사중복사업 자격에 해당되지 않는 자입니다. 그러나 예외적으로 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 제공이 가능합니다. 가사·간병 방문지원사업 이용자는 노인맞춤돌봄서비스의 유사중복사업 자격 해당자에 속합니다. 따라서 사용자가 노인맞춤돌봄서비스를 신청하려면 시장·군수·구청장이 서비스가 필요하다고 인정해야 합니다.'

In [ ]:
# 사용자 질문
question3 = "난 1962년생이고, 차상위계층이야. 노인맞춤돌봄서비스를 신청하고 싶어"

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
new_response = qa_chain.invoke({"context": context, "question": question3})

new_response

'노인맞춤돌봄서비스를 신청할 수 있습니다. 서비스 신청 자격에 따르면, 65세 이상 차상위계층이 서비스 대상에 해당되기 때문입니다. 서비스는 읍·면·동 행정복지센터를 방문하여 신청할 수 있습니다.'

In [ ]:
# 사용자 질문
question4 = "난 1963년생이고 노인맞춤돌봄서비스를 신청하고 싶어." # SolarAPI는 2023년을 기준으로 계산하는듯함.

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
new_response = qa_chain.invoke({"context": context, "question": question4})

new_response

'노인맞춤돌봄서비스를 신청하려면 65세 이상 국민기초생활수급자, 차상위계층 또는 기초연금수급자 자격에 해당되어야 합니다. 그러나 1963년생이시라면 올해 만 60세이시므로, 현재 노인맞춤돌봄서비스 신청자격에 해당되지 않습니다. 서비스 신청자격을 갖추기 위해서는 5년이 지나야 합니다.'

In [ ]:
# 사용자 질문
question5 = "난 1959년생이고 기초연금을 받고 있어. 장기요양 등급 유효기간이 만료되었는데 노인맞춤돌봄서비스를 신청할 수 있어?"

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
new_response = qa_chain.invoke({"context": context, "question": question5})

new_response

'예, 노인맞춤돌봄서비스를 신청할 수 있습니다. 문서에 따르면, 장기요양 등급 유효기간 만료자도 노인맞춤돌봄서비스를 신청할 수 있습니다. 단, 신청 가능하고 노인장기요양보험 등급 고지가 요청되며, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우에 예외적으로 신청이 가능합니다. 또한, 신청 당시 본인 질병이나 사유 등으로 장기요양 신청이 어려운 경우, 해당 사유를 기재한 장기요양정보제공신청서를 통해 확인 시 노인맞춤돌봄서비스를 제공받을 수 있습니다.'

In [ ]:
# 음성방언 - 혼저옵서예. 우리 어멍이 이시민 만 예순다섯 살 훨씬 넘으시고, 기초연금도 받엉 계십디다. 혼자 살암신디 몸도 안 좋아신게, 노인맞춤돌봄서비스 좀 신청허멍 받앙볼 수 있을까요?
# STT - 혼자 없어예 우리 어멍이 이십인 만 예순다섯 살 훨씬 넘으시고 기초연금도 받은 게 쉽디다. 혼자 살았은디 몸도 안 좋아지신 게 노인맞춤 돌봄 서비스를 좀 신청어멍 받아볼 수 있을까요

nlp_trans = "저 혼자 살고 있어요. 저희 어머니는 예순다섯 살이 훨씬 넘으셨고, 기초연금도 받고 계세요. 혼자 지내셨는데 건강도 안 좋아지셔서, 노인맞춤 돌봄 서비스를 신청해서 받을 수 있을까요?"

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
response = qa_chain.invoke({"context": context, "question": nlp_trans})

response

'문서에 따르면, 노인맞춤돌봄서비스의 신청자격은 65세 이상 국민기초생활수급자, 차상위계층 또는 기초연금수급자로서 유사중복사업 자격에 해당되지 않는 자입니다. 단, 시장·군수·구청장이 서비스가 필요하다고 인정하는 경우 예외적으로 제공이 가능합니다. \n\n질문자의 어머니는 65세 이상이며 기초연금을 받고 계시지만, 문서에 따르면 기초연금수급자 중 유사중복사업 자격 해당자가 아닌 경우에 서비스가 제공됩니다. 질문자의 어머니가 다른 유사중복사업(장기요양, 가사·간병 방문지원, 보훈재가복지서비스 등)을 이용 중이지 않다면, 건강 상태가 서비스 필요성으로 인정될 경우 노인맞춤돌봄서비스를 받을 수 있을 가능성이 있습니다. \n\n최종적인 서비스 제공 여부는 시장·군수·구청장이 결정하므로, 자세한 사항은 해당 지역의 행정복지센터에 문의하여 상담을 받아보시는 것이 좋습니다.'

In [ ]:
# 음성방언 - 혼저옵서예. 우리 어멍이 이시민 만 예순다섯 살 훨씬 넘으시고, 기초연금도 받엉 계십디다. 혼자 살암신디 몸도 안 좋아신게, 노인맞춤돌봄서비스 좀 신청허멍 받앙볼 수 있을까요?
# STT - 혼자 없어예 우리 어멍이 이십인 만 예순다섯 살 훨씬 넘으시고 기초연금도 받은 게 쉽디다. 혼자 살았은디 몸도 안 좋아지신 게 노인맞춤 돌봄 서비스를 좀 신청어멍 받아볼 수 있을까요

nlp_trans = "저 혼자 살고 있어요. 저희 어머니는 예순다섯 살이 훨씬 넘으셨고, 기초연금도 받고 계세요. 혼자 지내셨는데 건강도 안 좋아지셔서, 노인맞춤 돌봄 서비스를 신청해서 받을 수 있을까요?"

# 관련 문서 검색
docs = retriever.invoke(question)

# 문서 내용을 하나로 합침
context = "\n\n".join([doc.page_content for doc in docs])

# LLMChain 실행
response = qa_chain.invoke({"context": context, "question": nlp_trans})

response